In [ ]:
import os
from astropy.io import fits
import pandas as pd
from datetime import datetime

In [6]:


# Reference time for conversion
tref = datetime(2017, 1, 1)

def check_solar_angle(fits_file):

    """ Check if the SOLARANG value in the FITS header is less than 90 """
    with fits.open(fits_file) as hdul:
        solar_ang = hdul[1].header['SOLARANG']
    # return solar_ang < 90
    return True

def check_rate_in_interval(lc_file, tstart, tstop):
    with fits.open(lc_file) as hdul:  # Replace 'path_to_your_file.lc' with the path to your .lc file
        data = hdul[1].data
    df = pd.DataFrame(data)

In [7]:
def process_data(year, month, date, start_time, end_time, base_path='/home/baadalvm/MIDTERM_CODE/data/'):
    """ Process the data for a given year, month, and date """
    cla_path = os.path.join(base_path, f'cla/data/calibrated/{year}/{month:02d}/{date:02d}')
    xsm_path = os.path.join(base_path, f'xsm/data/{year}/{month:02d}/{date:02d}')

    # Convert start and end time from input to seconds since tref
    start_time_str = f"{year}-{month:02d}-{date:02d}T{start_time[:2]}:{start_time[2:4]}:{start_time[4:]}.000"
    end_time_str = f"{year}-{month:02d}-{date:02d}T{end_time[:2]}:{end_time[2:4]}:{end_time[4:]}.000"
    input_tstart = (datetime.strptime(start_time_str, '%Y-%m-%dT%H:%M:%S.%f') - tref).total_seconds()
    input_tstop = (datetime.strptime(end_time_str, '%Y-%m-%dT%H:%M:%S.%f') - tref).total_seconds()

    # List all FITS files in the CLA date directory
    fits_files = [os.path.join(cla_path, f) for f in os.listdir(cla_path) if f.endswith('.fits')]
    print("a")
    for fits_file in fits_files:
        with fits.open(fits_file) as hdul:
            # Extract start and end time from the FITS header
            startime = hdul[1].header['STARTIME']
            endtime = hdul[1].header['ENDTIME']
            # print(startime)
            
            # Convert start and end times to seconds since tref
            tstart = (datetime.strptime(startime, '%Y-%m-%dT%H:%M:%S.%f') - tref).total_seconds()
            tstop = (datetime.strptime(endtime, '%Y-%m-%dT%H:%M:%S.%f') - tref).total_seconds()
        # print("b")
        # Check if file duration lies within the specified input interval
        if tstart >= input_tstart and tstop <= input_tstop:
            print("c")
            # Check solar angle
            if check_solar_angle(fits_file):
                print("d")
                # Locate the .lc file
                lc_name = f'ch2_xsm_{year:04d}{month:02d}{date:02d}_v1_level2.lc'
                lc_file = os.path.join(xsm_path, 'calibrated', lc_name) 

                # Check for a flare within the interval using the .lc file
                if check_rate_in_interval(lc_file, tstart, tstop):
                    print("e")
                    raw_folder_path = os.path.join(xsm_path, 'raw')
                    calibrated_folder_path = os.path.join(xsm_path, 'callibrated')

                    # Execute pipeline2.py with the required arguments
                    print(raw_folder_path)
                    print(calibrated_folder_path)
                    print(fits_file)
                    os.system(f"/usr/bin/python3 /home/baadalvm/MIDTERM_CODE/src/pipeline2.py {fits_file} {raw_folder_path} {calibrated_folder_path}")

In [11]:
process_data(2024,7,1,"100215","100228")

a
c
d


In [3]:
tref = datetime(2017, 1, 1)
start = "2024-07-01T10:02:15.000"
end = "2024-07-01T10:02:28.000"
tstart = (datetime.strptime(start, '%Y-%m-%dT%H:%M:%S.%f')-tref).total_seconds()
tend = (datetime.strptime(end, '%Y-%m-%dT%H:%M:%S.%f')-tref).total_seconds()

In [4]:
with fits.open('/home/baadalvm/MIDTERM_CODE/data/xsm/data/2024/07/01/calibrated/ch2_xsm_20240701_v1_level2.lc') as hdul:  # Replace 'path_to_your_file.lc' with the path to your .lc file
    

AttributeError: 'Series' object has no attribute 'byteswap'

In [18]:
df['RATE']

0        3781.627686
1        3746.636230
2        3938.148438
3        3667.842529
4        3973.721924
            ...     
10524    3408.155029
10525    3437.526611
10526    3453.349121
10527    3457.578857
10528    3475.097168
Name: RATE, Length: 10529, dtype: float32

In [28]:
df[(df['TIME'] >= tstart) & (df['TIME'] <= tend)]

ValueError: Big-endian buffer not supported on little-endian compiler

In [29]:
tstart

236599335.0

In [32]:
filtered_df = df[(df['TIME'] >= tstart) & (df['TIME'] <= tend)]

ValueError: Big-endian buffer not supported on little-endian compiler